In [1]:
# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import os

# change to path
PATH='/content/drive/My Drive/Colab Notebooks/christies'
os.chdir(PATH)

In [0]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import gc
import re

from datetime import datetime
from tqdm import tqdm

import warnings
warnings.simplefilter(action='ignore')

## Read in

In [0]:
df = pd.read_csv('dataset/post_war.csv')

In [8]:
df.head()

,title,auction_id,auction_loc,sale_tot,parent,date,item_id,item_name,item_past,img_url,item_des,item_prov,est_price,real_price
0,Post-War and Contemporary Art Day Auction,17194,London,"GBP 12,663,250",/Post-War-and-Contemporary-Art-28093,26 Jun 2019,101,Still Life\n,\nNicolas Party (b.1980)\r\n\n,https://www.christies.com/img/LotImages/2019/C...,"[""Nicolas Party (b.1980)\nStill Life\r\nsigned...","Private Collection, Italy.","GBP 20,000 - GBP 40,000","GBP 150,000"
1,Post-War and Contemporary Art Day Auction,17194,London,"GBP 12,663,250",/Post-War-and-Contemporary-Art-28093,26 Jun 2019,102,It's My Pot of Gold\n,\nShara Hughes (b. 1981)\r\n\n,https://www.christies.com/img/LotImages/2019/C...,"[""Shara Hughes (b. 1981)\nIt's My Pot of Gold\...","Private Collection, London.\r\nAnon. sale, Phi...","GBP 25,000 - GBP 35,000","GBP 125,000"
2,Post-War and Contemporary Art Day Auction,17194,London,"GBP 12,663,250",/Post-War-and-Contemporary-Art-28093,26 Jun 2019,103,(black 14)\n,\nShio Kusaka (B. 1972)\r\n\n,https://www.christies.com/img/LotImages/2019/C...,['Shio Kusaka (B. 1972)\n(black 14)\r\nstonewa...,"The Modern Institute, Glasgow.\r\nAcquired fro...","GBP 7,000 - GBP 9,000","GBP 11,875"
3,Post-War and Contemporary Art Day Auction,17194,London,"GBP 12,663,250",/Post-War-and-Contemporary-Art-28093,26 Jun 2019,104,(black 13)\n,\nShio Kusaka (B. 1972)\r\n\n,https://www.christies.com/img/LotImages/2019/C...,['Shio Kusaka (B. 1972)\n(black 13)\r\nstonewa...,"The Modern Institute, Glasgow.\r\nAcquired fro...","GBP 4,000 - GBP 6,000","GBP 5,625"
4,Post-War and Contemporary Art Day Auction,17194,London,"GBP 12,663,250",/Post-War-and-Contemporary-Art-28093,26 Jun 2019,105,(ink 3)\n,\nShio Kusaka\r\n\n,https://www.christies.com/img/LotImages/2019/C...,['Shio Kusaka\n(ink 3)\r\nporcelain\r\n9 ½ x 4...,"Blum & Poe, Los Angeles. \r\nAcquired from the...","GBP 5,000 - GBP 8,000","GBP 22,500"


In [9]:
df.isna().sum()

title              0
auction_id         0
auction_loc        0
sale_tot           0
parent             0
date               0
item_id            0
item_name       1245
item_past          0
img_url         3191
item_des           0
item_prov      10861
est_price        667
real_price       669
dtype: int64

## Cleaning

In [0]:
# converts auction_id to int
df['auction_id'] = df['auction_id'].astype(int)

In [0]:
# spilts sales into currecy and value
def currency(s):
    return s.split(' ')[1]
  
def sales_value(s):
    return int(s.split(' ')[2].replace(',', ''))
  
df['currency'] = df['sale_tot'].apply(currency)
df['sale'] = df['sale_tot'].apply(sales_value)

In [0]:
# clean parent
df['parent'] = df['parent'].apply(lambda o: o.split('/')[1])

In [0]:
# split dates into start and end
def clean_date(s):
    dates = s.split()
    
    # if auction is more than a day
    if len(dates) == 6:
        start = ' '.join([dates[0], dates[1], dates[5]])
        end = ' '.join([dates[3], dates[4], dates[5]])
        
    elif len(dates) == 5:
        start = ' '.join([dates[0], dates[-2], dates[-1]])
        end = ' '.join([dates[-3], dates[-2], dates[-1]])
    
    # auction is only a day
    else:
        start = ' '.join(dates)
        end = start
        
    return datetime.strptime(start, "%d %b %Y"), datetime.strptime(end, "%d %b %Y")
    
  
start_end = df['date'].apply(clean_date)

df['auction_start'] = [start_end[i][0] for i in range(len(start_end))]
df['auction_end'] = [start_end[i][1] for i in range(len(start_end))]

In [0]:
# # convert item_id to int type
def clean_item_id(s):
    if len(s) > 7: # long descriptions
        return int(s.split('\n')[1].split(' ')[-1])
    elif not s.isalnum(): # contains char
        return int(s.split(' ')[0])
    else:
        return int(s)

df['item_id'] = df['item_id'].apply(clean_item_id)

In [0]:
# converts nan back to empty str
df['item_name'].fillna('', inplace=True)

df['item_name'] = df['item_name'].apply(lambda o: o.strip('\n').strip('\r'))

In [0]:
# converts nan back to empty str
df['item_past'].fillna('', inplace=True)

df['item_past'] = df['item_past'].apply(lambda o: o.strip('\n').strip('\r'))

In [0]:
# converts nan back to empty str
df['item_prov'].fillna('', inplace=True)

df['item_prov'] = df['item_prov'].apply(lambda o: o.strip('\n').strip('\r').replace('\r\n', ' '))

In [0]:
# converts nan back to empty str
df['item_des'].fillna('', inplace=True)

def clean_des(s):
    try:
        return s.strip('[').strip(']').replace('\\n', ' ').replace('\\r\\', ' ').replace('\\r', ' ').replace('\\', ' ')
    except:
        return s
      
df['item_des'] = df['item_des'].apply(clean_des)

In [0]:
# spilts realised price into currecy and value
def real_currency(s):
    try:
        return s.split(' ')[0]
    except:
        return s
  
def real_price(s):
    try:
        return int(s.split(' ')[1].replace(',', ''))
    except:
        return s
      
def converts(s):
    try:
        return int(s)
    except:
        return np.nan           
  
df['real_currency'] = df['real_price'].apply(real_currency)
df['realized_price'] = df['real_price'].apply(real_price)
df['realized_price'] = df['realized_price'].apply(converts)

In [0]:
def est_price(s):
    try:
        prices = s.split('-')
        low_est = prices[0].split(' ')[1].replace(',', '')
        high_est = prices[-1].split(' ')[-1].replace(',', '')
        return int(low_est), int(high_est)
    except:
        return (np.nan, np.nan)
      
low_high_est = df['est_price'].apply(est_price)

df['low_est'] = [low_high_est[i][0] for i in range(len(low_high_est))]
df['high_est'] = [low_high_est[i][1] for i in range(len(low_high_est))]

## Image name
Create image name.

In [21]:
def create_img_name(idx):
    return ('0' * (5 - len(str(idx)))) + str(idx) + '.jpg'
  
df['idx'] = df.index

df['img'] = df['idx'].apply(create_img_name)

df.drop('idx', axis=1, inplace=True)

df.head()

,title,auction_id,auction_loc,sale_tot,parent,date,item_id,item_name,item_past,img_url,item_des,item_prov,est_price,real_price,currency,sale,auction_start,auction_end,real_currency,realized_price,low_est,high_est,img
0,Post-War and Contemporary Art Day Auction,17194,London,"GBP 12,663,250",Post-War-and-Contemporary-Art-28093,26 Jun 2019,101,Still Life,Nicolas Party (b.1980),https://www.christies.com/img/LotImages/2019/C...,"""Nicolas Party (b.1980) Still Life signed and...","Private Collection, Italy.","GBP 20,000 - GBP 40,000","GBP 150,000",GBP,12663250,2019-06-26,2019-06-26,GBP,150000.0,20000.0,40000.0,00000.jpg
1,Post-War and Contemporary Art Day Auction,17194,London,"GBP 12,663,250",Post-War-and-Contemporary-Art-28093,26 Jun 2019,102,It's My Pot of Gold,Shara Hughes (b. 1981),https://www.christies.com/img/LotImages/2019/C...,"""Shara Hughes (b. 1981) It's My Pot of Gold s...","Private Collection, London. Anon. sale, Philli...","GBP 25,000 - GBP 35,000","GBP 125,000",GBP,12663250,2019-06-26,2019-06-26,GBP,125000.0,25000.0,35000.0,00001.jpg
2,Post-War and Contemporary Art Day Auction,17194,London,"GBP 12,663,250",Post-War-and-Contemporary-Art-28093,26 Jun 2019,103,(black 14),Shio Kusaka (B. 1972),https://www.christies.com/img/LotImages/2019/C...,'Shio Kusaka (B. 1972) (black 14) stoneware ...,"The Modern Institute, Glasgow. Acquired from t...","GBP 7,000 - GBP 9,000","GBP 11,875",GBP,12663250,2019-06-26,2019-06-26,GBP,11875.0,7000.0,9000.0,00002.jpg
3,Post-War and Contemporary Art Day Auction,17194,London,"GBP 12,663,250",Post-War-and-Contemporary-Art-28093,26 Jun 2019,104,(black 13),Shio Kusaka (B. 1972),https://www.christies.com/img/LotImages/2019/C...,'Shio Kusaka (B. 1972) (black 13) stoneware ...,"The Modern Institute, Glasgow. Acquired from t...","GBP 4,000 - GBP 6,000","GBP 5,625",GBP,12663250,2019-06-26,2019-06-26,GBP,5625.0,4000.0,6000.0,00003.jpg
4,Post-War and Contemporary Art Day Auction,17194,London,"GBP 12,663,250",Post-War-and-Contemporary-Art-28093,26 Jun 2019,105,(ink 3),Shio Kusaka,https://www.christies.com/img/LotImages/2019/C...,'Shio Kusaka (ink 3) porcelain 9 ½ x 4 ¾ x 4...,"Blum & Poe, Los Angeles. Acquired from the ab...","GBP 5,000 - GBP 8,000","GBP 22,500",GBP,12663250,2019-06-26,2019-06-26,GBP,22500.0,5000.0,8000.0,00004.jpg


## Download images

In [0]:
import urllib.request 
import requests

!mkdir images

In [0]:
# slower!

# def create_img_name(idx, url):
#     fn = 'images/' + ('0' * (5 - len(str(idx)))) + str(idx) + '.jpg'
#     urllib.request.urlretrieve(url, fn)

# for i in tqdm(range(len(img_urls))):
#     create_img_name(i, img_urls[i])

In [24]:
img_urls = df['img_url'].tolist()

len(img_urls)

51534

In [0]:
import multiprocessing
from io import BytesIO
from urllib import request
from PIL import Image

In [0]:
# from https://www.kaggle.com/lyakaap/fast-resized-image-download-python-3

# set files and dir
OUT_DIR = 'dataset/images'  

# preferences
TARGET_SIZE = 256  # image resolution to be stored
IMG_QUALITY = 90  # JPG quality
NUM_WORKERS = 2  # Num of CPUs

In [0]:
def download_image(key_url):
    url = key_url
    
    try:
        key = key_url.split('/')[-1]
    except:
        return 1
           
    filename = os.path.join(OUT_DIR,key)

    if os.path.exists(filename): # image exists
        return 0

    try:
        response = request.urlopen(url) # download image
        image_data = response.read()
    except:
        return 1

    try:
        pil_image = Image.open(BytesIO(image_data)) # parse image
    except:
        return 1

    try:
        pil_image = pil_image.convert('RGB') # convert image to RGB
    except:
        return 1

    try:
        pil_image = pil_image.resize((TARGET_SIZE, 
                                      TARGET_SIZE)) # resize
    except:
        return 1

    try:
        pil_image.save(filename, format='JPEG', 
                       quality=IMG_QUALITY) # save as jpg
    except:
        return 1

    return 0


def loader(df):
    if not os.path.exists(OUT_DIR):
        os.mkdir(OUT_DIR)

    url_list = df['img_url'].tolist()
    
    pool = multiprocessing.Pool(processes=NUM_WORKERS)
    
    failures = sum(tqdm(pool.imap_unordered(download_image, url_list),
                        total=len(url_list)))
    print('Total number of download failures:', failures)
    pool.close()
    pool.terminate()

In [0]:
loader(df)

  0%|          | 3/51534 [01:45<553:58:44, 38.70s/it]

In [0]:
gc.collect()

In [0]:
# missing = []

# def create_img_name(idx, url):
#     filename = 'dataset/images/' + ('0' * (5 - len(str(idx)))) + str(idx) + '.jpg'
#     if (not os.path.isfile(filename)):
#         try:
#             r = requests.get(url, allow_redirects=True)            
#             open(filename, 'wb').write(r.content)
#         except Exception as e:
#             missing.append(idx)

In [0]:
# for i in tqdm(range(len(img_urls))):
#     create_img_name(i, img_urls[i])

In [0]:
len(os.listdir('dataset/images'))

48343

In [0]:
# with open('missing.txt', 'w') as f:
#     for item in missing:
#         f.write("%s\n" % item)

File size.

In [0]:
# def convert_bytes(num):
#     for x in ['bytes', 'KB', 'MB', 'GB', 'TB']:
#         if num < 1024.0:
#             return "%3.1f %s" % (num, x)
#         num /= 1024.0

# def file_size():
#     os.chdir('dataset/images')
#     print('Image file size is', convert_bytes(sum(os.path.getsize(f) for f in os.listdir('.') if os.path.isfile(f))))
#     os.chdir('../..')

# file_size()

## Exchange rate
Get exchange rate.

In [0]:
# !unzip full_webstats_xru_current_dataflow_csv.zip

# https://www.bis.org/statistics/xrusd.htm

In [0]:
# fx = pd.read_csv('dataset/WEBSTATS_XRU_CURRENT_DATAFLOW_csv_col.csv')

# fx = fx[fx['Frequency'] == 'Monthly']

# fx.reset_index(inplace=True, drop=True)

# fx.columns[3502:3663] # '2006-01'
# fx.columns[3662] # '2019-05'

# fx = fx.iloc[:,np.r_[3:5,3502:3663]] # only countries and monthly

# fx = fx.iloc[slice(0,len(fx), 2)]

# fx_ = fx.groupby('CURRENCY').mean().reset_index()

# fx_.head()

# currs = ['EUR', 'GBP', 'CNY', 'HKD']

# fx_ = fx_[fx_['CURRENCY'].isin(currs)]

# fx_.to_csv('dataset/currency_06_19.csv', index=False)

Creates an `exchange_rate` column.

In [0]:
# converts all currency to USD
fx = pd.read_csv('dataset/currency_06_19.csv')

# copy exchange rate of 2019-05 over to 2019-06
fx['2019-06'] = fx['2019-05']

# make currency the index
fx.set_index('CURRENCY', inplace=True)

fx.head()

,2006-01,2006-02,2006-03,2006-04,2006-05,2006-06,2006-07,2006-08,2006-09,2006-10,2006-11,2006-12,2007-01,2007-02,2007-03,2007-04,2007-05,2007-06,2007-07,2007-08,2007-09,2007-10,2007-11,2007-12,2008-01,2008-02,2008-03,2008-04,2008-05,2008-06,2008-07,2008-08,2008-09,2008-10,2008-11,2008-12,2009-01,2009-02,2009-03,2009-04,...,2016-03,2016-04,2016-05,2016-06,2016-07,2016-08,2016-09,2016-10,2016-11,2016-12,2017-01,2017-02,2017-03,2017-04,2017-05,2017-06,2017-07,2017-08,2017-09,2017-10,2017-11,2017-12,2018-01,2018-02,2018-03,2018-04,2018-05,2018-06,2018-07,2018-08,2018-09,2018-10,2018-11,2018-12,2019-01,2019-02,2019-03,2019-04,2019-05,2019-06
CURRENCY,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
CNY,8.066496,8.051049,8.035039,8.015585,8.015362,8.006742,7.990387,7.972740,7.933299,7.901946,7.863102,7.822403,7.788257,7.750110,7.738153,7.724155,7.674170,7.632342,7.575348,7.573048,7.522452,7.502531,7.420435,7.371391,7.240733,7.165917,7.074267,6.998984,6.972039,6.898371,6.836474,6.851819,6.837663,6.836149,6.829188,6.856521,6.835803,6.836795,6.836298,6.831558,...,6.506778,6.478554,6.530542,6.592455,6.677528,6.648173,6.673025,6.729756,6.842736,6.922789,6.895511,6.872602,6.897014,6.891173,6.885102,6.808887,6.773221,6.670848,6.568458,6.625751,6.621600,6.596276,6.427250,6.322437,6.321415,6.300631,6.374249,6.466318,6.718020,6.848650,6.855803,6.921122,6.939536,6.886526,6.788843,6.738166,6.712520,6.717222,6.860944,6.860944
EUR,0.834360,0.845747,0.840195,0.822991,0.790984,0.799077,0.797296,0.788536,0.793573,0.800094,0.782414,0.761715,0.774051,0.769451,0.759391,0.743472,0.743734,0.749198,0.732303,0.737683,0.723667,0.706565,0.684112,0.689500,0.682785,0.680908,0.646242,0.636876,0.643972,0.643217,0.634366,0.668320,0.696398,0.752317,0.786119,0.744539,0.755758,0.782384,0.767233,0.758556,...,0.901096,0.881915,0.884194,0.890637,0.903479,0.891959,0.891907,0.907028,0.926297,0.948623,0.942201,0.939657,0.936005,0.932691,0.904551,0.890555,0.868785,0.847010,0.839343,0.850663,0.852091,0.844888,0.819874,0.809916,0.810642,0.814625,0.846623,0.856336,0.855745,0.865963,0.857746,0.870814,0.879781,0.878432,0.875934,0.880981,0.884778,0.889840,0.894093,0.894093
GBP,0.566827,0.572105,0.573511,0.566133,0.535125,0.542856,0.542298,0.528202,0.530446,0.533315,0.523267,0.509256,0.510366,0.510931,0.513708,0.502632,0.504291,0.503512,0.491724,0.497492,0.495723,0.489311,0.482812,0.494632,0.507728,0.509204,0.499121,0.504663,0.509169,0.508952,0.502925,0.529656,0.556342,0.591440,0.652498,0.672608,0.693946,0.693783,0.705064,0.680507,...,0.702987,0.698722,0.687614,0.704188,0.759878,0.762791,0.760146,0.810832,0.804643,0.801046,0.811219,0.801255,0.810213,0.791172,0.773732,0.781223,0.769885,0.771780,0.750922,0.757681,0.756511,0.745730,0.724229,0.715919,0.715687,0.710468,0.742731,0.752594,0.759279,0.776637,0.765824,0.768700,0.775233,0.788590,0.776097,0.768790,0.759331,0.766854,0.779445,0.779445
HKD,7.754232,7.759879,7.759589,7.756535,7.754259,7.763697,7.773755,7.776517,7.782509,7.785921,7.782261,7.773117,7.800060,7.811557,7.813423,7.815377,7.818766,7.814080,7.819375,7.815679,7.782764,7.754508,7.778037,7.797991,7.804308,7.797278,7.782315,7.791977,7.799739,7.807285,7.800047,7.808289,7.787801,7.758940,7.750966,7.750534,7.756627,7.753668,7.750343,7.750284,...,7.761778,7.756598,7.763622,7.762245,7.756966,7.756270,7.756305,7.757317,7.756171,7.758992,7.756612,7.759952,7.766634,7.774475,7.787299,7.798931,7.809191,7.821627,7.813490,7.805704,7.805138,7.813236,7.819532,7.822038,7.840837,7.848431,7.848803,7.847229,7.847832,7.849635,7.837537,7.837905,7.829816,7.820106,7.842002,7.847850,7.849423,7.844967,7.848236,7.848236


In [0]:
df['ym'] = df['auction_end'].apply(lambda o: o.strftime('%Y-%m'))

def find_fx(curr, ym):
    if curr == 'USD': return 1
    return fx.loc[curr, ym]
  
# get the exchange rate 
# during the time auction ended
df['exchange_rate'] = df.apply(lambda x: find_fx(x['currency'], x['ym']), 
                               axis=1)

## Anomaly artist


It was noticed that some items do not have an artist, for example,

<a href="https://imgur.com/4KGhQMK"><img src="https://i.imgur.com/4KGhQMK.png" title="source: imgur.com" width="500px" /></a>

Instead, these items' `artist` is, in fact, the `item_name`. Items alike were tricky to look for among the 50,000 odd items. But a trend was observed that such items have all uppercase titles and starts with the determiner "A". Hence,  `spaCy` was used to look for such items by checking for determiner.

After retrieving the indices of such items, swap the `item_name` with `artist`.

In [0]:
df['item_past'].fillna('', inplace=True)

# remove date
df['artist'] = df['item_past'].apply(lambda o: o.split('(')[0])

In [0]:
import spacy

nlp = spacy.load('en_core_web_sm')

docs = nlp.pipe(df['item_past'])

In [0]:
errors = []

i = 0 # track index of weird items 

for doc in tqdm(docs):
    if doc.text.isupper(): # text is all uppercase 
        for token in doc:
            if token.pos_ == 'DET': # if there is a determiner              
                # for checking later
                errors.append(i)
                
                # swap here
                temp = df.iloc[i].copy()
                df['item_name'].iloc[i] = df['artist'].iloc[i]
                df['artist'].iloc[i] = temp['item_name']
                
                break
    i += 1
    
print('Number of wrong item\'s label:', len(errors))    

51534it [12:27, 68.98it/s] 

Number of wrong item's label: 3484


In [0]:
df['artist'].fillna('', inplace=True)

# converts artist to title case
def convert_title(s):
    try: 
      return s.title()
    except:
      return s
  
df['artist'] = df['artist'].apply(convert_title)

(df['artist'].value_counts()
             .tail()
             .reset_index()
             .rename(columns={'index': 'artist',
                              'artist': 'count'}))

,artist,count
0,Augustus Edwin John,1
1,Paire De Tables De Salon Modernes,1
2,Tete De Belier En Bronze,1
3,First Quarter 20Th Century,1
4,Dick Bruna,1


Although not perfect, this approach still manages to rule out many of the anomalies.

## Reorder 
Reordering columns.

In [0]:
df.columns

Index(['title', 'auction_id', 'auction_loc', 'sale_tot', 'parent', 'date',
       'item_id', 'item_name', 'item_past', 'img_url', 'item_des', 'item_prov',
       'est_price', 'real_price', 'currency', 'sale', 'auction_start',
       'auction_end', 'real_currency', 'realized_price', 'low_est', 'high_est',
       'img', 'ym', 'exchange_rate', 'artist'],
      dtype='object')

In [0]:
clean_df = df[['title', 'auction_id', 'auction_loc', 
               'sale', 'currency',
               'parent', 
               'auction_start', 'auction_end',
               'item_id', 'item_name', 'item_past', 
               'artist', 'item_des', 'item_prov', 
               'img_url', 'img',
               'exchange_rate',
               'low_est', 'high_est',
               'realized_price']].copy()

clean_df.head()

,title,auction_id,auction_loc,sale,currency,parent,auction_start,auction_end,item_id,item_name,item_past,artist,item_des,item_prov,img_url,img,exchange_rate,low_est,high_est,realized_price
0,Post-War and Contemporary Art Day Auction,17194,London,12663250,GBP,Post-War-and-Contemporary-Art-28093,2019-06-26,2019-06-26,101,Still Life,Nicolas Party (b.1980),Nicolas Party,"""Nicolas Party (b.1980) Still Life signed and...","Private Collection, Italy.",https://www.christies.com/img/LotImages/2019/C...,00000.jpg,0.779445,20000.0,40000.0,150000.0
1,Post-War and Contemporary Art Day Auction,17194,London,12663250,GBP,Post-War-and-Contemporary-Art-28093,2019-06-26,2019-06-26,102,It's My Pot of Gold,Shara Hughes (b. 1981),Shara Hughes,"""Shara Hughes (b. 1981) It's My Pot of Gold s...","Private Collection, London. Anon. sale, Philli...",https://www.christies.com/img/LotImages/2019/C...,00001.jpg,0.779445,25000.0,35000.0,125000.0
2,Post-War and Contemporary Art Day Auction,17194,London,12663250,GBP,Post-War-and-Contemporary-Art-28093,2019-06-26,2019-06-26,103,(black 14),Shio Kusaka (B. 1972),Shio Kusaka,'Shio Kusaka (B. 1972) (black 14) stoneware ...,"The Modern Institute, Glasgow. Acquired from t...",https://www.christies.com/img/LotImages/2019/C...,00002.jpg,0.779445,7000.0,9000.0,11875.0
3,Post-War and Contemporary Art Day Auction,17194,London,12663250,GBP,Post-War-and-Contemporary-Art-28093,2019-06-26,2019-06-26,104,(black 13),Shio Kusaka (B. 1972),Shio Kusaka,'Shio Kusaka (B. 1972) (black 13) stoneware ...,"The Modern Institute, Glasgow. Acquired from t...",https://www.christies.com/img/LotImages/2019/C...,00003.jpg,0.779445,4000.0,6000.0,5625.0
4,Post-War and Contemporary Art Day Auction,17194,London,12663250,GBP,Post-War-and-Contemporary-Art-28093,2019-06-26,2019-06-26,105,(ink 3),Shio Kusaka,Shio Kusaka,'Shio Kusaka (ink 3) porcelain 9 ½ x 4 ¾ x 4...,"Blum & Poe, Los Angeles. Acquired from the ab...",https://www.christies.com/img/LotImages/2019/C...,00004.jpg,0.779445,5000.0,8000.0,22500.0


## Save

In [0]:
clean_df.to_csv('dataset/post_war_art_clean.csv', index=False)